#### This is my Coursera Assignement on Clustering

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

!conda install -c conda-forge beautifulsoup4 --yes # install BeautifulSoup 4

import requests # library to handle requests
print('Libraries Ready')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  35.81 MB/s
Libraries Ready


In [2]:
# assign Wiki page to a variable
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# assign the result of a request to a variable

page = requests.get(url)


In [4]:
# import Beautiful Soup into the Python console:

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.text, 'html.parser')


In [5]:
# find the table object
table_content = soup.find(class_="wikitable sortable")

#find all the rows
tb_rows = table_content.find_all("tr")


In [6]:
data = []  

# insert the table data in a list object
for row in tb_rows:
    cols = row.find_all('td')
    
    # insert data to list if its not emty
    if len(cols) != 0:
        cols = [ele.text.strip() for ele in cols]
        
        # insert only if 2nd element of list (Borough) is not equal to Not Assigned
        if cols[1] != 'Not assigned':
            data.append([ele for ele in cols if ele])

In [7]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
My_DF = pd.DataFrame(data, columns=column_names)

# replace Not assigned Neighborhoods with Borough name
My_DF.Neighborhood.replace('Not assigned',My_DF.Borough,inplace=True)

# Combine Neighborhoods for the same Postal Code
My_DF = My_DF.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()


In [8]:
My_DF.shape 

(103, 3)

In [9]:
# Get the Geo coordinate csv data
!wget -q -O 'geo_coord.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')


Data downloaded!


In [20]:
# Load data to a data frame
Coord_DF = pd.read_csv('geo_coord.csv')
Coord_DF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
# Combine the Geo Coordinate Dataframe to the earlier Dataframe
Merged_DF = pd.merge(left=My_DF,right=Coord_DF, left_on='PostalCode', right_on='Postal Code')
Merged_DF.head()

# Drop the Postal Code column from tje Geo Coordinate dataframe
Merged_DF = Merged_DF.drop('Postal Code', axis=1)
Merged_DF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
